In [1]:
!pip install pyspark py4j

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.8/310.8 MB 4.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.4.0-py2.py3-none-any.whl size=311317130 sha256=f8441e544e3d6ace6ab93e1372068102117f76396fb47be171b8b7966542661f
  Stored in directory: /root/.cache/pip/wheels/7b/1b/4b/3363a1d04368e7ff0d408e57ff57966fcdf00583774e761327
Successfully built pyspark


In [2]:
from pyspark.sql import SparkSession

In [3]:
from pyspark.ml.regression import LinearRegression

In [4]:
spark = SparkSession.builder.appName('hnd_lr').getOrCreate()

In [5]:
data = spark.read.csv('cruise_ship_info.csv',inferSchema=True,header=True)

In [6]:
data.show()

+-----------+-----------+---+------------------+----------+------+------+-----------------+----+
|  Ship_name|Cruise_line|Age|           Tonnage|passengers|length|cabins|passenger_density|crew|
+-----------+-----------+---+------------------+----------+------+------+-----------------+----+
|    Journey|    Azamara|  6|30.276999999999997|      6.94|  5.94|  3.55|            42.64|3.55|
|      Quest|    Azamara|  6|30.276999999999997|      6.94|  5.94|  3.55|            42.64|3.55|
|Celebration|   Carnival| 26|            47.262|     14.86|  7.22|  7.43|             31.8| 6.7|
|   Conquest|   Carnival| 11|             110.0|     29.74|  9.53| 14.88|            36.99|19.1|
|    Destiny|   Carnival| 17|           101.353|     26.42|  8.92| 13.21|            38.36|10.0|
|    Ecstasy|   Carnival| 22|            70.367|     20.52|  8.55|  10.2|            34.29| 9.2|
|    Elation|   Carnival| 15|            70.367|     20.52|  8.55|  10.2|            34.29| 9.2|
|    Fantasy|   Carnival| 23| 

In [9]:
data.printSchema()

root
 |-- Ship_name: string (nullable = true)
 |-- Cruise_line: string (nullable = true)
 |-- Age: integer (nullable = true)
 |-- Tonnage: double (nullable = true)
 |-- passengers: double (nullable = true)
 |-- length: double (nullable = true)
 |-- cabins: double (nullable = true)
 |-- passenger_density: double (nullable = true)
 |-- crew: double (nullable = true)



In [8]:
from pyspark.ml.feature import StringIndexer

In [10]:
indexer = StringIndexer(inputCol='Cruise_line',outputCol='cruise_cat')
indexed = indexer.fit(data).transform(data)
indexed.head(1)

[Row(Ship_name='Journey', Cruise_line='Azamara', Age=6, Tonnage=30.276999999999997, passengers=6.94, length=5.94, cabins=3.55, passenger_density=42.64, crew=3.55, cruise_cat=16.0)]

In [11]:
from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import VectorAssembler

In [13]:
indexed.columns

['Ship_name',
 'Cruise_line',
 'Age',
 'Tonnage',
 'passengers',
 'length',
 'cabins',
 'passenger_density',
 'crew',
 'cruise_cat']

In [16]:
assembler = VectorAssembler(inputCols=['Age','Tonnage',
 'passengers',
 'length',
 'cabins',
 'passenger_density',
 'cruise_cat'
],outputCol='features')

In [17]:
output = assembler.transform(indexed)

In [20]:
output.select('features','crew').show()

+--------------------+----+
|            features|crew|
+--------------------+----+
|[6.0,30.276999999...|3.55|
|[6.0,30.276999999...|3.55|
|[26.0,47.262,14.8...| 6.7|
|[11.0,110.0,29.74...|19.1|
|[17.0,101.353,26....|10.0|
|[22.0,70.367,20.5...| 9.2|
|[15.0,70.367,20.5...| 9.2|
|[23.0,70.367,20.5...| 9.2|
|[19.0,70.367,20.5...| 9.2|
|[6.0,110.23899999...|11.5|
|[10.0,110.0,29.74...|11.6|
|[28.0,46.052,14.5...| 6.6|
|[18.0,70.367,20.5...| 9.2|
|[17.0,70.367,20.5...| 9.2|
|[11.0,86.0,21.24,...| 9.3|
|[8.0,110.0,29.74,...|11.6|
|[9.0,88.5,21.24,9...|10.3|
|[15.0,70.367,20.5...| 9.2|
|[12.0,88.5,21.24,...| 9.3|
|[20.0,70.367,20.5...| 9.2|
+--------------------+----+
only showing top 20 rows



In [21]:
train_data,test_data = output.select('features','crew').randomSplit([0.7,0.3])

In [22]:
train_data.describe().show()

+-------+------------------+
|summary|              crew|
+-------+------------------+
|  count|               114|
|   mean| 7.775526315789485|
| stddev|3.6356786787412263|
|    min|              0.59|
|    max|              21.0|
+-------+------------------+



In [23]:
test_data.describe().show()

+-------+------------------+
|summary|              crew|
+-------+------------------+
|  count|                44|
|   mean| 7.842499999999998|
| stddev|3.1743493198844965|
|    min|              0.88|
|    max|             12.53|
+-------+------------------+



In [24]:
lr = LinearRegression(labelCol='crew')

In [25]:
lr_Model = lr.fit(train_data)

In [26]:
test_results = lr_Model.evaluate(test_data)

In [27]:
test_results.residuals.show()

+--------------------+
|           residuals|
+--------------------+
| -1.2640444012109562|
|  1.7611144145767783|
| -0.8879764598313296|
| -0.8879764598313296|
| 0.20663258035790122|
| -0.5776236791525324|
| -1.5240245444588734|
|  1.0583588153483046|
| -0.7284135505510037|
| -0.9846393469090309|
|  2.2282562345316457|
| 0.06699248790278922|
|  0.9490622153157027|
|  0.9376922319122958|
| -0.5197568299756359|
|  0.8518307422005087|
|-0.03963585673688996|
|  0.9662065937668007|
|  0.9833509722179006|
| 0.14817786169713187|
+--------------------+
only showing top 20 rows



In [28]:
test_results.rootMeanSquaredError

0.8178150817099615

In [29]:
train_data.describe().show()

+-------+------------------+
|summary|              crew|
+-------+------------------+
|  count|               114|
|   mean| 7.775526315789485|
| stddev|3.6356786787412263|
|    min|              0.59|
|    max|              21.0|
+-------+------------------+



In [30]:
test_results.r2

0.9320819797760262

In [31]:
test_results.meanSquaredError

0.6688215078722711

In [32]:
test_results.meanAbsoluteError

0.6492106613990255

In [33]:
from pyspark.sql.functions import corr

In [34]:
data.select(corr('crew','passengers')).show()

+----------------------+
|corr(crew, passengers)|
+----------------------+
|    0.9152341306065384|
+----------------------+



In [35]:
data.select(corr('crew','cabins')).show()

+------------------+
|corr(crew, cabins)|
+------------------+
|0.9508226063578497|
+------------------+



In [36]:
unlabeled_data = test_data.select('features')

In [37]:
unlabeled_data.show()

+--------------------+
|            features|
+--------------------+
|[5.0,86.0,21.04,9...|
|[5.0,122.0,28.5,1...|
|[6.0,30.276999999...|
|[6.0,30.276999999...|
|[6.0,110.23899999...|
|[6.0,112.0,38.0,9...|
|[7.0,89.6,25.5,9....|
|[9.0,113.0,26.74,...|
|[10.0,105.0,27.2,...|
|[10.0,138.0,31.14...|
|[10.0,151.4,26.2,...|
|[11.0,58.6,15.66,...|
|[11.0,91.0,20.32,...|
|[11.0,108.977,26....|
|[12.0,58.6,15.66,...|
|[12.0,88.5,21.24,...|
|[12.0,90.09,25.01...|
|[12.0,91.0,20.32,...|
|[13.0,91.0,20.32,...|
|[13.0,101.509,27....|
+--------------------+
only showing top 20 rows



In [38]:
predictions = lr_Model.transform(unlabeled_data)

In [39]:
predictions.show()

+--------------------+------------------+
|            features|        prediction|
+--------------------+------------------+
|[5.0,86.0,21.04,9...| 9.264044401210956|
|[5.0,122.0,28.5,1...| 4.938885585423222|
|[6.0,30.276999999...| 4.437976459831329|
|[6.0,30.276999999...| 4.437976459831329|
|[6.0,110.23899999...|11.293367419642099|
|[6.0,112.0,38.0,9...|11.477623679152533|
|[7.0,89.6,25.5,9....|11.394024544458873|
|[9.0,113.0,26.74,...|11.321641184651696|
|[10.0,105.0,27.2,...|11.408413550551003|
|[10.0,138.0,31.14...| 12.83463934690903|
|[10.0,151.4,26.2,...|10.301743765468354|
|[11.0,58.6,15.66,...|  7.53300751209721|
|[11.0,91.0,20.32,...| 9.040937784684298|
|[11.0,108.977,26....|11.062307768087704|
|[12.0,58.6,15.66,...| 7.519756829975636|
|[12.0,88.5,21.24,...|  9.43816925779949|
|[12.0,90.09,25.01...|  8.71963585673689|
|[12.0,91.0,20.32,...|   9.0237934062332|
|[13.0,91.0,20.32,...|   9.0066490277821|
|[13.0,101.509,27....|11.351822138302868|
+--------------------+------------